# Tutorial: The Canonical HANK Model & Fiscal Policy

This file has blanks that we will fill in the tutorial together.

This notebook aims to provide a step-by-step tutorial on how to set up the canonical HANK model from scratch using `sequence-jacobian` and then proceeding to compute some simple impulse responses.

The first step is to import packages we'll need for this notebook.

In [ ]:
import numpy as np  # numpy helps us perform linear algebra calculations
import matplotlib.pyplot as plt  # helps us plot
import sequence_jacobian as sj  # SSJ will allow us to define blocks, models, compute IRFs, etc

Before we start, we'll define some basic model parameters we need below. We store them in the `calibration` dictionary:

In [ ]:
calibration = {'eis': 0.5,  # EIS
               'rho_e': 0.9,  # Persistence of idiosyncratic productivity shocks
               'sd_e': 0.92,  # Standard deviation of idiosyncratic productivity shocks
               'G': 0.2,  # Government spending
               'B': 0.8,  # Government debt
               'Y': 1.,  # Output
               'min_a': 0.,  # Minimum asset level on the grid
               'max_a': 1_000,  # Maximum asset level on the grid
               'n_a': 200,  # Number of asset grid points
               'n_e': 10}  # Number of productivity grid points

## Building the canonical HA Model

### Defining the `HetBlock`

First, we will set up the `HetBlock` that represents our heterogeneous household problem.

For the sake of clarity, we directly write these in the code block below, but since it's a really standard `Block`, we can also import it directly from within `sequence-jacobian` by calling `from sj.hetblocks.hh_sim import hh_init, hh`.

 - `hh_init` initializies the values for the backward iteration, which is `Va`, the derivative of the value function with respect to assets, defined over a joint grid of income and asset states. We always write the grids as (income state, asset state).
 - `hh` is a single backward step of the endogenous gridpoints method, covered in an earlier session.
 - the decorator `@sj.het` turns the subsequent function definition into a `HetBlock`. We'll have similar decorators below to define blocks.

We just created our first block! Let's check it out.

We see that some of these inputs are vectors, such as `a_grid` and `z`. We typically try to define them within the `HetBlock` itself. We can do so by attaching a `hetinput` function that constructs these inputs "in-house" so to speak.

A `hetinput` is a function, which takes either scalar- or vector-valued inputs that are generated from other upstream `hetinput`s, and yields vector-valued outputs that feed into its associated `HetBlock`.

- `make_grids` instantiates the productivity and assets grids:
    - productivity grid `e_grid` is from the Rouwenhorst method for discretizing AR(1) processes.
    - asset grid `a_grid` uses a non-uniform spacing concentrating grid points near the bottom, where policies exhibit the most curvature.

- `income` converts the productivity grid `e_grid` into the post-tax income grid `z`.

Note: in the `.markov_rouwenhorst` function, we normalize $\mathbb{E}[e] = 1$, which is consistent with the exogenous labor supply $N = 1$ in this model.

We define `hh_extended` as a `HetBlock` that has these additional `hetinputs`.

Let's see what the new block does.

We see that the inputs changed: Now `a_grid` and `e_grid` are no longer inputs. Instead, `Z`, total after tax income, is among the inputs. In fact, `Z` will be the only input that will change in GE in our experiments below.

### Defining the `SimpleBlock`s

Next, we will specify two `SimpleBlock`s that will allow us to close our model: a fiscal policy (government) block and a market clearing block.

Here, we define the fiscal block in terms of government bonds `B`.

Now we can put these `Block`s together to create a model!

Let's see what kind of animal `ha` is.

A model is really a `CombinedBlock`, which, like our $H$ function in lecture, itself has inputs and outputs.

Many of the inputs are model parameters, such as `eis`, `beta`, `rho_e`, etc. But some are actually aggregates we want to solve ("unknowns") for or shock ("exogenous inputs"). One nice feature in SSJ is that we don't have to distinguish the two at this point. This gives us flexibility to shock whatever we want later on.

Some of the outputs are "targets", such as `goods_mkt` or `asset_mkt`. By Walras' law, it is sufficient to impose either `asset_mkt` or `goods_mkt`.

## Calibrating a steady state

To solve for a steady state, we need to specify the unknowns that we are calibrating to satisfy the targets of the model. We can omit one of them (`goods_mkt` in this case) for now by Walras' law. We will check its value afterward to ensure our model is correctly specified. If this test fails, we made a mistake when setting up the model.

Because we want to fix the steady state interest rate `r` at a pre-specified value, to satisfy 1) `asset_mkt`, we will calibrate `beta`.

We could try to find the steady state ourselves:

Or we do this automatically, by solving for the `beta` that sets `asset_mkt` to zero.

Now let's verify that Walras law holds

### Matching MPCs

One important lesson from today's classes was that MPC's are really important, more important than the wealth distribution itself to accurately capture the aggregate dynamics of heterogeneous-agent models. So could we try to match equilibrium MPCs here, too?

To do this, we need to compute MPCs, but those depend on the policy function `c` that lives within the `HetBlock` for now! We see this by checking `hh_extended`'s outputs:

To compute MPCs, we can define a `hetoutput` function:

A `hetoutput` is a function, which takes scalar- and vector-valued inputs provided in the `calibration` or computed within the `HetBlock` it is attached to, and produces vector-valued outputs. A common use case for `hetoutput`s is to compute a quantity that is not necessary as a step in the backward iteration of the `HetBlock` itself but is useful as a target in the model DAG or as an output the user cares to inspect.

Other examples of `hetoutput` functions are functions that compute, e.g. moments of the wealth distribution.

In [ ]:
def compute_weighted_mpc(c, a, a_grid, r, e_grid):
    """Approximate mpc out of wealth, with symmetric differences where possible, exactly setting mpc=1 for constrained agents."""
    mpc = np.empty_like(c)
    post_return = (1 + r) * a_grid
    mpc[:, 1:-1] = (c[:, 2:] - c[:, 0:-2]) / (post_return[2:] - post_return[:-2])
    mpc[:, 0] = (c[:, 1] - c[:, 0]) / (post_return[1] - post_return[0])
    mpc[:, -1] = (c[:, -1] - c[:, -2]) / (post_return[-1] - post_return[-2])
    mpc[a == a_grid[0]] = 1
    mpc = mpc * e_grid[:, np.newaxis]
    return mpc




What did this do? Let's see how it affected `hh_extended`.

Now we have an MPC variable as output! Let's check what it is in our current calibration by evaluating `hh_extended` at the `ss` dict.

That's already very close, but let's say we want to match it perfectly. One way we can do so is by modifying `B`, the overall amount of liquidity, to hit it perfectly.

Why? Choosing `B` sets the amount of liquidity available to households to ensure themselves against their idiosyncratic risk. As the amount of liquidity in the economy goes down, the cross-sectional distribution of households shifts toward the constrained asset level, raising their MPCs.

Let's proceed to computing transitional dynamics with this new steady state

Notice that `ss` here can be used to inspect the steady state we have computed. It works much like a `dict` in python.

One fun thing you can do here is plot the wealth distribution, which is stored in `ss.internals`.

## Transitional dynamics

Now that we have a steady state, let's compute some simple impulse responses

First, we will look at the response of output to a fully tax-financed increase in government spending of 1% of GDP, with persistence $\rho_G = 0.8$

What are the unknowns and targets we should specify to solve for this IRF? Let's investigate inputs and outputs of `ha` to figure this out.

The only unknown is `Y` here. We have two `target`s, but only need one, so let's use `asset_mkt` for now. (`goods_mkt` is similar.)

To do this, we call the `.solve_impulse_linear` method on the `ha` model object, providing as input arguments: `ss`, the list of unknowns, the list of targets, and the dict of shocks. You can inspect the contents of the returned `ImpulseDict` object, by indexing into it as you would a `dict` object.

Let's see what we got. First, let's just look at the numbers.

Those are the first few entries of our `Y` response. Let's plot it!

To do so, we use a predefined simple plotting function.

In [ ]:
def show_irfs(irfs_list, variables, labels=[" "], ylabel=r"Percentage points (dev. from ss)", T_plot=50, figsize=(18, 6)):
    if len(irfs_list) != len(labels):
        labels = [" "] * len(irfs_list)
    n_var = len(variables)
    fig, ax = plt.subplots(1, n_var, figsize=figsize, sharex=True)
    for i in range(n_var):
        # plot all irfs
        for j, irf in enumerate(irfs_list):
            ax[i].plot(100 * irf[variables[i]][:50], label=labels[j])
        ax[i].set_title(variables[i])
        ax[i].set_xlabel(r"$t$")
        if i==0:
            ax[i].set_ylabel(ylabel)
        ax[i].legend()
    plt.show()

Now we're ready to plot IRFs:

What if we want to make the increase in `G` entirely debt financed initially?

This means we need to feed in a simultaneous shock to `B`, that initially increases by the same amount as `G`.

Let's compare IRFs across the two specifications.

That's interesting! The output effect is stronger with deficit financed spending. Why? Let's look at consumption

... because there was no effect on consumption from a balanced budget spending shock.

## Comparisons with RA and TA

Great, so we solved and understood our first HA model together!

Next, let's try to compare the propagation in HA with that in RA and TA models. To do so, we write `SimpleBlock`s that define the household problems for the representative and two-agent versions of the simple heterogeneous agent model we implemented above. What's nice is that we won't have to rewrite any other block!

We begin with the RA model. Here, we notice that the propagation is entirely determined by the Euler equation, which is recursive, so we'll have to handle that with a `SolvedBlock`.

One big problem we notice here already: When $\beta = 1/(1+r)$, neither `C` nor `A` are determined! This is because the model has an infinitely elastic steady state savings supply curve.

So in the steady state, we'll have to solve for `C` and `A` directly.

We do something similar for the TA model.

We redefine `beta` in the calibration, and match the same `B` that we had in the HA model. Finally we pin down `C`  and `A`  to be consistent with the budget constraint and asset market clearing. Using the `dissolve` keyword argument turns the `SolvedBlock`s into `SimpleBlock`s for the purpose of steady state evaluation, i.e. to "promote" `C`, `A` from being unknowns at their individual block level to the model level.

First we will plot the impulse response to the same government spending shock that we used above for the HA model to ensure that the balanced budget responses are indeed the same across RA, TA, HA

Awesome! So we see that the HA model generates a large positive consumption response, much larger than the TA model. This is because the HA model generates higher iMPCs off the diagonal.

## Exercises

### Exercise 1: Sketch the DAG

Sketch the DAG that underlies the `ha` model.

### Exercise 2: Other shocks in the canonical HANK model

Feed in a negative 1% real interest rate shock into the model. This is an accommodative monetary policy shock. What happens in the three models? How is the MPC affected by monetary polic?

Do the same for other shocks:
 - shocks to `beta`?
 - shocks to risk aversion (negative shocks to `eis`)? This only makes sense in the HA model.
 - shocks to idiosyncratic risk (`sd_e`)? This only makes sense in the HA model.

### Exercise 3: Direct and indirect effects in monetary policy transmission

Consider our accommodative monetary policy shock from exercise 1. Focus on the household side. Can you use the equilibrium paths of `r` and `Z` to decompose how important each channel is in monetary transmission?
_(Hint: You can use `hh.impulse_linear` to simulate individual blocks)_